# Script de scraper pour collecter les recettes de marmiton.org

**Note :** Dans ce Notebook nous ne collecterons que $100$ recettes du site marmiton.org, à titre d'exemple. En effet, lancer le scraper sur l'ensemble des recettes du site prenant un peu de temps on ne le lancera pas ici sur les $30000$ recettes.

On importe les librairies dont nous aurons besoin.

In [36]:
#Pour collecter les pages web
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests

from numpy import *
import time 

#Pour gérer les json
import json
import jsonpickle

list_url= []
list_new_url=[]
list_recette=[]

#Dans cette exemple nous allons récupérer les recettes de plats principaux
url = 'http://www.marmiton.org/recettes/recherche.aspx?aqt=plat-principal'
prefix = 'http://www.marmiton.org'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'lxml')
new_url = soup.find_all('nav')[-2].find('li',attrs={'class','next-page'}).find('a').get('href')
list_url.append(url)

Ci-dessus, nous initialisons le scraper en lui donnant l'url de la page de laquelle il commencera à scraper.
Ensuite, on parcourt toutes les url de pages listant des recettes (chaque page contient $15$ urls de recettes que nous récupérerons ensuite).

In [37]:
#Pour prendre toute les url de pages suivantes de manière générale
#while type(new_url)==str:
#    list_url.append(new_url)
#    page = requests.get(prefix+new_url)
#    soup = BeautifulSoup(page.content, 'lxml')
#    new_url = soup.find_all('nav')[-2].find('li',attrs={'class','next-page'}).find('a').get('href')
#    print(len(list_url))

#Connaissant la structure des urls du site, on récupérera beaucoup plus rapidement les urls avec :
gen ='/recettes/recherche.aspx?aqt=plat-principal&start='
for i in range (12,26460,12):
    list_url.append(prefix + gen + str(i))
print('Nombre d\'urls récupérées : ', len(list_url))

Nombre d'urls récupérées :  2205


**Notes : ** $26460$ est ici approximativement le nombre de recettes disponibles le jour ou nous lançons le programme, ce nombre est susceptible de changer, c'est à vérifier sur le site.

On collecte toutes les urls de recettes ainsi contenues dans les urls précédentes.

In [51]:
#Pour collecter toutes les url de recettes (ici on ne collectera que les 100 premières)
    
for i in range(len(list_url)) :
    if len(list_recette) < 100 :
        page = requests.get(list_url[i])
        soup = BeautifulSoup(page.content, 'lxml')
        descrp= soup.find_all('a',attrs={'class','recipe-card'})
        for j in range(len(descrp)):
            urlf = descrp[j].get('href')
            if urlf not in list_recette :
                list_recette.append(urlf)
print('%i urls de recettes récupérées' % len(list_recette))

105 urls de recettes récupérées


Après avoir récupéré toutes les urls contenant des urls de recettes la collecte peut commencer. En parcourant le code source des pages au préalable, nous avons remarqué que celles-ci contiennent un  *.json* contenant toutes les informations de chaque recette. Après avoir localisé ce *.json* nous le collectons pour chaque recette. Chaque *.json* de recette est ainsi ajouté et sauvé dans un fichier général que nous appellons *plats.json* ici et qui regroupera tout les plats du site et leurs informations.

Le site marmiton étant un peu à cheval sur le nombre de requêtes envoyées par une même adresse IP, on décide d'également ajouter un $\texttt{sleep}$ au programme, pour lui permettre de dormir $15$ minutes entre deux rappels à l'ordre.

In [42]:
start = time.time()

saveFile = open('plats.json', 'w', encoding='utf-8')
#Parcourir toutes les url de recettes et sauvegarder toutes les infos en json
for i in range(len(list_recette)):
    page = requests.get(prefix + list_recette[i])
    soup = BeautifulSoup(page.content, 'lxml')
    jsonpickle.set_encoder_options('json', ensure_ascii=False)
    try :
        text = json.loads(str(soup.find('script', type='application/ld+json'))[58:-22])
    except : 
        print('erreur à l\'url : ',prefix+list_recette[i])
        time.sleep(900)

    saveFile.write(jsonpickle.encode(text, unpicklable=False) + '\n')
    print(i+1,'/',len(list_recette),'recettes ajoutées')

saveFile.close()
elapsed_time=round(time.time()-start,2)
print('Fini en',elapsed_time,'s')

1 / 105 recettes ajoutées
2 / 105 recettes ajoutées
3 / 105 recettes ajoutées
4 / 105 recettes ajoutées
5 / 105 recettes ajoutées
6 / 105 recettes ajoutées
7 / 105 recettes ajoutées
8 / 105 recettes ajoutées
9 / 105 recettes ajoutées
10 / 105 recettes ajoutées
11 / 105 recettes ajoutées
12 / 105 recettes ajoutées
13 / 105 recettes ajoutées
14 / 105 recettes ajoutées
15 / 105 recettes ajoutées
16 / 105 recettes ajoutées
17 / 105 recettes ajoutées
18 / 105 recettes ajoutées
19 / 105 recettes ajoutées
20 / 105 recettes ajoutées
21 / 105 recettes ajoutées
22 / 105 recettes ajoutées
23 / 105 recettes ajoutées
24 / 105 recettes ajoutées
25 / 105 recettes ajoutées
26 / 105 recettes ajoutées
27 / 105 recettes ajoutées
28 / 105 recettes ajoutées
29 / 105 recettes ajoutées
30 / 105 recettes ajoutées
31 / 105 recettes ajoutées
32 / 105 recettes ajoutées
33 / 105 recettes ajoutées
34 / 105 recettes ajoutées
35 / 105 recettes ajoutées
36 / 105 recettes ajoutées
37 / 105 recettes ajoutées
38 / 105 r

Le *.json* étant sauvegardé nous pouvons maintenant l'ouvrir pour en lire le contenu.

In [52]:
#Pour chercher les recettes dans le json
plats=[]
for line in open('plats.json','r'):
    plats.append(json.loads(line))
#On affiche les 10 premiers noms de recettes
for i in range(10):
    print(plats[i]['name'])

Soupe Chinoise en plat principal
Plat de côtes de porc grillé
Haricots plats à la tomate
Plat-de-côte, sauce tomate, poivrons.
Sablés façon oeuf au plat à la mangue
Roulés de sole aux haricots plats et crème légère à la vanille
Saucisse fraiche en sauce tomate et haricots plats
pavé de cabillaud aux haricots plats et fenouil sauce citronné
Haricots plats (coco) pimentés
Cocotte terre/mer à la Licorne Bio


Grâce à ce *.json* on peut par exemple prendre une recette et afficher son nom ainsi que les ingrédients qui la compose.

In [50]:
print('Recette : ')
print()
print(plats[0]['name'])
print()
print('Ingrédients nécessaires : ')
print()
print(plats[0]['recipeIngredient'])

Recette : 

Soupe Chinoise en plat principal

Ingrédients nécessaires : 

['8 cuisse de poulet', '2 boîte crabe', '30 crevette', '6 oeuf', 'champignon noir', '2 boîte pousse de bambou', 'vermicelles', '400 g germes de soja', 'ciboule', 'chou chinois', 'soja', 'pâte', '1 bouquet coriandre', '1 pot concentré pour soupe chinoise', '2.5 l eau']
